In [ ]:
using JuMP
using GLPK

In [ ]:
function exJuMP()
    m = JuMP.Model()

    @variable(m, x1 >= 0)
    @variable(m, x2 >= 0)

    @objective(m,  Max, 3 * x1 + 5 * x2)

    @constraint(m, r01,     x1          <= 4)
    @constraint(m, r02,          2 * x2 <= 12)
    @constraint(m, r03, 3 * x1 + 2 * x2 <= 18)

    JuMP.set_optimizer(m, GLPK.Optimizer)
    JuMP.optimize!(m)
    println()
    
    @show JuMP.termination_status(m)
    @show JuMP.primal_status(m)
    @show JuMP.dual_status(m)
    println()
    
    report = lp_sensitivity_report(m)
    
    @show JuMP.objective_value(m)
    for i in JuMP.all_variables(m)
        xval = JuMP.value(i)
        dx_lo, dx_hi = report[i]
        fo = JuMP.objective_function(m)
        c = JuMP.coefficient(fo, i)
        rc = JuMP.reduced_cost(i)
        println("$i=$xval \t -> rc = $rc \t Δ: ($dx_lo, $dx_hi) \t -> α: ($(c+dx_lo):$(c+dx_hi))")
    end

    println()
    
    @show JuMP.dual_objective_value(m)
    for i in list_of_constraint_types(m)
        if i[1] == VariableRef continue end

        for j in JuMP.all_constraints(m, i[1], i[2])
            ys = JuMP.shadow_price(j) # shadow_price vs dual
            b = JuMP.normalized_rhs(j)

            dRHS_lo, dRHS_hi = report[j]
            println("$j \t -> shadow_price: $ys \t -> Δ: ($dRHS_lo, $dRHS_hi) \t -> α: ($(b+dRHS_lo):$(b+dRHS_hi))")
        end
    end

    return
end

In [ ]:
exJuMP()

In [ ]:
function exDual()
    m = JuMP.Model()

    @variable(m, y1 >= 0)
    @variable(m, y2 >= 0)
    @variable(m, y3 >= 0)

    @objective(m,  Min,  4 * y1 + 12 * y2 + 18*y3)

    @constraint(m, r01,  1 * y1           +  3*y3 >=  3)
    @constraint(m, r02,            2 * y2 +  2*y3 >=  5)

    JuMP.set_optimizer(m, GLPK.Optimizer)
    JuMP.optimize!(m)
    
    @show JuMP.termination_status(m)
    @show JuMP.objective_value(m)
    @show JuMP.value.([y1 y2 y3])

    return
end

In [ ]:
exDual()